### Chargement du jeu de données nettoyé

Importer les bibliothèques nécessaires et charger le fichier `data_drop_duplicates.csv`,
en convertissant automatiquement la colonne `time` en datetime. Vérification des informations générales du DataFrame.


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Chemin vers votre fichier nettoyé
file_path = 'data_drop_duplicates.csv'

# Charger le CSV en convertissant la colonne 'time' en datetime dès le début
try:
    df = pd.read_csv(file_path, parse_dates=['time'])
    print("Fichier CSV chargé avec succès.")
    print("La colonne 'time' a été automatiquement convertie en datetime.")
except FileNotFoundError:
    print(f"Erreur : Le fichier '{file_path}' n'a pas été trouvé. Vérifiez le chemin.")
    # Si le fichier n'est pas trouvé, on arrête le script pour éviter des erreurs
    exit()

# Afficher les informations pour vérifier le type de la colonne 'time'
print("\nInformations sur le DataFrame :")
df.info()


### 🧹 Nettoyage des doublons à la minute

Nous procédons ici au nettoyage détaillé des doublons temporels avec trois étapes principales :

#### 🔹 Étape 1 : Détection et suppression des doublons profonds
- 🔍 Identifier les lignes totalement identiques (même minute + mêmes valeurs sur tous les capteurs).
- 🗑️ Supprimer ces doublons en conservant la première occurrence.
- ✅ Vérifier que tous les doublons profonds ont bien été supprimés.

#### 🔹 Étape 2 : Identification des quasi-doublons
- ⏱️ Compter le nombre d’enregistrements par minute.
- ⚠️ Repérer les minutes avec plusieurs valeurs différentes.
- 📝 Extraire ces lignes dans `df_quasi_duplicates` pour une analyse plus approfondie.

#### 🔹 Étape 3 : Nettoyage final
- ✂️ Supprimer la colonne temporaire `time_minute`.
- 📊 Vérifier la taille finale du DataFrame principal `df`.

Cette étape permet de préparer le DataFrame pour la **fusion conditionnelle** des mesures similaires.


In [ ]:
print("--- Début du processus de nettoyage des doublons à la minute ---")
print(f"Taille initiale du DataFrame : {df.shape}")

# =============================================================================
# === ÉTAPE 1 : DÉTECTION ET SUPPRESSION DES DOUBLONS PROFONDS ===
# =============================================================================
print("\n--- Étape 1: Traitement des doublons profonds ---")

# Création de la colonne de travail 'time_minute'
df['time_minute'] = df['time'].dt.floor('min')

# Définition du sous-ensemble de colonnes pour la vérification
# (la minute + tous les capteurs, en ignorant les secondes de 'time')
columns_to_check = df.columns.drop('time').tolist()

# 1a. Détecter le nombre de doublons profonds AVANT suppression
num_deep_duplicates_before = df.duplicated(subset=columns_to_check).sum()

if num_deep_duplicates_before > 0:
    print(f"Trouvé {num_deep_duplicates_before} doublon(s) profond(s). Suppression en cours...")

    # 1b. Appliquer la suppression
    rows_before_deep_dup_removal = len(df)
    df.drop_duplicates(subset=columns_to_check, keep='first', inplace=True)
    rows_after_deep_dup_removal = len(df)

    # 1c. Vérification et rapport
    print(f"{rows_before_deep_dup_removal - rows_after_deep_dup_removal} ligne(s) de doublons profonds ont été supprimées.")

    # Re-vérification pour s'assurer que tout a été supprimé
    num_deep_duplicates_after = df.duplicated(subset=columns_to_check).sum()
    if num_deep_duplicates_after == 0:
        print("✅ Vérification : Suppression des doublons profonds réussie.")
    else:
        print("⚠️ Erreur : Il reste des doublons profonds après la tentative de suppression.")
else:
    print("✅ Aucun doublon profond trouvé.")

print(f"Taille du DataFrame après l'étape 1 : {df.shape}")

# =============================================================================
# === ÉTAPE 2 : IDENTIFICATION DES QUASI-DOUBLONS RESTANTS ===
# =============================================================================
print("\n--- Étape 2: Identification des minutes avec des valeurs différentes ---")

# La colonne 'time_minute' existe déjà, nous pouvons la réutiliser.

# 2a. Compter le nombre d'occurrences de chaque minute
minute_counts = df['time_minute'].value_counts()

# 2b. Identifier les minutes qui apparaissent encore plus d'une fois
# Ce sont, par définition, les cas où les valeurs des capteurs sont différentes.
quasi_duplicate_minutes = minute_counts[minute_counts > 1].index

if not quasi_duplicate_minutes.empty:
    print(f"{len(quasi_duplicate_minutes)} minutes contiennent encore des enregistrements multiples (avec des valeurs différentes).")

    # 2c. Extraire ces lignes pour une analyse future
    df_quasi_duplicates = df[df['time_minute'].isin(quasi_duplicate_minutes)].copy()

    # Trier pour la lisibilité
    df_quasi_duplicates.sort_values(by=['time_minute', 'time'], inplace=True)

    print(f"Un total de {len(df_quasi_duplicates)} lignes sont concernées.")
    print("Ces lignes sont stockées dans le DataFrame 'df_quasi_duplicates' pour une analyse plus approfondie.")
    # display(df_quasi_duplicates.head()) # Décommenter pour voir un aperçu
else:
    print("✅ Aucune minute avec des enregistrements multiples ne reste.")

# =============================================================================
# === ÉTAPE 3 : NETTOYAGE FINAL ===
# =============================================================================

# Supprimer la colonne de travail 'time_minute' du DataFrame principal
df.drop(columns=['time_minute'], inplace=True, errors='ignore')

print("\n--- Processus de nettoyage terminé ---")
print(f"Taille finale du DataFrame principal 'df' : {df.shape}")

### ⚗️ Fusion conditionnelle des enregistrements par minute

Après avoir identifié les quasi-doublons, nous fusionnons les mesures similaires pour chaque minute en suivant une logique précise :

#### 🔹 Étape 1 : Création des groupes temporels
- ⏱️ Créer la colonne `time_minute` (arrondie à la minute) pour regrouper les enregistrements.
- 🔢 Identifier toutes les colonnes numériques (`numeric_cols`) pour le calcul des statistiques.

#### 🔹 Étape 2 : Calcul des moyennes globales
- 📊 Calculer la **moyenne globale** pour chaque capteur afin de servir de référence pour le seuil de fusion.

#### 🔹 Étape 3 : Analyse des variations locales
- Min et max de chaque capteur pour chaque minute multi-enregistrement.
- Calcul de la **plage de variation** (spread) pour chaque groupe.
- Définir un **seuil global** basé sur 20% de la moyenne globale (`marge = 0.2`).
- Déterminer quelles minutes sont **fusionnables** (variation ≤ seuil pour toutes les colonnes).

#### 🔹 Étape 4 & 5 : Séparer et traiter les données
- 📌 Minutes non touchées : enregistrements uniques ou groupes à forte variation.
- 🔗 Minutes à fusionner : calcul de la moyenne pour chaque capteur et conservation du premier `time`.

#### 🔹 Étape 6 : Combiner et finaliser
- 🔄 Concaténer les DataFrames fusionnés et inchangés pour créer `df_cleaned`.
- 🔍 Trier par `time` pour la cohérence chronologique.
- ✅ Vérifier et afficher le nombre de minutes encore contenant des enregistrements multiples.

Cette étape permet de **réduire les doublons subtils** tout en conservant la précision et la cohérence des mesures.


In [ ]:
# Créer la colonne 'time_minute' pour identifier les groupes
df['time_minute'] = df['time'].dt.floor('min')

marge = 0.2

# Lister les colonnes numériques
numeric_cols = df.select_dtypes(include=np.number).columns

# --- Étape 1 : Calcul des moyennes GLOBALES (la modification clé) ---
print("\n--- Calcul des moyennes globales pour chaque capteur ---")
global_means = df[numeric_cols].mean()

# --- Étape 2 : Identifier les groupes et calculer leurs stats locales ---

# On ne travaille que sur les minutes ayant plus d'un enregistrement
minute_counts = df['time_minute'].value_counts()
multi_record_minutes = minute_counts[minute_counts > 1].index
df_multi = df[df['time_minute'].isin(multi_record_minutes)]

# Calculer les stats MIN et MAX pour ces groupes
stats_per_minute = df_multi.groupby('time_minute')[numeric_cols].agg(['min', 'max'])

# Extraire chaque statistique pour des calculs plus clairs
mins = stats_per_minute.xs('min', axis=1, level=1)
maxs = stats_per_minute.xs('max', axis=1, level=1)

# --- Étape 3 : Appliquer la condition de fusion avec le seuil GLOBAL ---

# Calculer la plage de variation (spread) pour chaque groupe
spread = maxs - mins

# Calculer le seuil en utilisant les moyennes GLOBALES
# C'est la ligne de code qui change la logique !
threshold = marge * global_means

# La condition est vraie si la plage est inférieure ou égale au seuil global.
# Pandas aligne automatiquement le DataFrame 'spread' et la Série 'threshold' par colonne.
condition_met = spread <= threshold

# Une minute est "fusionnable" si la condition est vraie pour TOUTES ses colonnes
is_mergeable = condition_met.all(axis=1)

# Extraire les listes de minutes pour chaque catégorie
minutes_to_merge = is_mergeable[is_mergeable].index
minutes_to_keep = is_mergeable[~is_mergeable].index

print(f"\nAnalyse de {len(multi_record_minutes)} minutes avec plusieurs enregistrements :")
print(f"  -> {len(minutes_to_merge)} minutes seront fusionnées (variation < 20% de la moyenne globale).")
print(f"  -> {len(minutes_to_keep)} minutes seront conservées telles quelles (variation > 20% de la moyenne globale).")

# --- Étape 4 & 5 : Séparer et traiter les données (identique à avant) ---

# 1. Données qui ne sont pas touchées (enregistrements uniques ou groupes à forte variation)
single_record_minutes = minute_counts[minute_counts == 1].index
minutes_untouched = single_record_minutes.union(minutes_to_keep)
df_untouched = df[df['time_minute'].isin(minutes_untouched)].copy()

# 2. Données à fusionner en calculant leur moyenne
df_to_merge = df[df['time_minute'].isin(minutes_to_merge)]

agg_dict = {col: 'mean' for col in numeric_cols}
agg_dict['time'] = 'first'
df_merged = df_to_merge.groupby('time_minute').agg(agg_dict)

# --- Étape 6 : Combiner et analyser le résultat (identique à avant) ---

# Préparer les DataFrames pour la concaténation
df_untouched.drop(columns=['time_minute'], inplace=True)
df_merged.reset_index(drop=True, inplace=True)

# Combiner pour créer le DataFrame final nettoyé
df_cleaned = pd.concat([df_untouched, df_merged], ignore_index=True)
df_cleaned.sort_values(by='time', inplace=True, ignore_index=True)

print(f"\nTaille du DataFrame après fusion conditionnelle : {df_cleaned.shape}")

# Compter les "doublons" restants dans le nouveau DataFrame
df_cleaned['time_minute'] = df_cleaned['time'].dt.floor('min')
final_minute_counts = df_cleaned['time_minute'].value_counts()
remaining_multi_records = final_minute_counts[final_minute_counts > 1]

print("\n" + "="*50)
print("RÉSULTAT FINAL")
print("="*50)
print(f"Nombre de minutes contenant encore des enregistrements multiples : {len(remaining_multi_records)}")

### 🕵️‍♂️ Analyse détaillée des minutes avec variation significative

Après la fusion conditionnelle, certaines minutes présentent encore des **écarts importants** entre les enregistrements.
Cette cellule permet de les isoler et de les examiner.

#### 🔹 Étape 1 : Isolation des lignes à analyser
- ⏱️ Identifier les minutes restantes avec plusieurs enregistrements (`remaining_multi_records`).
- 📂 Filtrer le DataFrame `df_cleaned` pour ne garder que ces lignes → `df_to_inspect`.

#### 🔹 Étape 2 : Préparation de l’affichage
- 🔀 Trier par `time_minute` et `time` pour regrouper les enregistrements d’une même minute.
- 👀 Configurer Pandas pour afficher toutes les colonnes et une largeur suffisante pour une lecture claire.

#### 🔹 Étape 3 : Affichage des résultats
- 📝 Afficher le nombre total de lignes et de minutes restantes.
- ⚠️ Identifier pour chaque minute les **colonnes problématiques** dont l’écart dépasse le seuil.
- 🗂️ Ajouter ces informations au DataFrame `df_to_inspect` dans la colonne `problem_columns`.

Cette étape fournit une **vue détaillée des anomalies restantes** et prépare les minutes problématiques pour une analyse ou un traitement complémentaire.


In [ ]:
# --- Étape 1 : Isoler les lignes à analyser ---

# Obtenir la liste des timestamps (minutes) qui nous intéressent
minutes_to_inspect = remaining_multi_records.index

# Filtrer le DataFrame 'df_cleaned' pour ne garder que les lignes correspondant à ces minutes
df_to_inspect = df_cleaned[df_cleaned['time_minute'].isin(minutes_to_inspect)].copy()

# --- Étape 2 : Préparer l'affichage pour une meilleure lisibilité ---

# Trier les lignes pour que les enregistrements d'une même minute soient regroupés
df_to_inspect.sort_values(by=['time_minute', 'time'], inplace=True)

# Pour s'assurer que toutes les 92 colonnes sont visibles et non tronquées
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000) # Augmenter la largeur d'affichage

# --- Étape 3 : Afficher le résultat ---

print("\n" + "="*80)
print("AFFICHAGE DES LIGNES POUR LES MINUTES AVEC UNE VARIATION SIGNIFICATIVE")
print("="*80)
print(f"Voici les {len(df_to_inspect)} lignes correspondant aux {len(remaining_multi_records)} minutes restantes :")

# --- Étape A (Rappel de la logique) : Déterminer les colonnes problématiques par minute ---
# (Ces variables 'spread' et 'threshold' doivent être disponibles depuis le script précédent)
is_problematic = spread > threshold

# --- Étape B : Pour chaque minute, créer une liste des noms de colonnes problématiques ---
# C'est la ligne clé : elle transforme la grille de True/False en une Série de listes de noms
problematic_cols_per_minute = is_problematic.apply(lambda row: row.index[row].tolist(), axis=1)

# --- Étape C : Ajouter cette information au DataFrame d'inspection ---
# On utilise .map() pour assigner la liste de colonnes à chaque ligne en fonction de sa minute
df_to_inspect['problem_columns'] = df_to_inspect['time_minute'].map(problematic_cols_per_minute)

display(df_to_inspect)

### 🔍 Analyse contextuelle des groupes problématiques

Cette étape permet d’examiner en détail les **groupes de deux lignes présentant une variation significative** pour chaque minute problématique.

#### 🔹 Vérifications initiales
- ⚠️ S’assurer que les DataFrames `df_cleaned` et `df_to_inspect` existent avant de commencer.
- 🔢 Identifier les colonnes numériques à analyser.

#### 🔹 Étape 1 : Calcul des moyennes globales
- 📊 Calculer **une seule fois** les moyennes globales pour chaque capteur (`global_means`) afin de les utiliser comme référence dans la comparaison.

#### 🔹 Étape 2 : Parcourir chaque groupe problématique
- ⏱️ Boucler sur chaque minute ayant deux enregistrements significatifs.
- ❌ Ignorer les groupes avec un nombre de lignes différent de 2.

#### 🔹 Étape 3 : Calcul des moyennes avant et après
- 🔹 Pour chaque ligne du groupe, calculer la moyenne des 10 lignes précédentes (`moyenne_avant`).
- 🔹 Calculer la moyenne des 10 lignes suivantes (`moyenne_apres`).
- Ces statistiques locales servent de **contexte** pour évaluer les écarts.

#### 🔹 Étape 4 : Préparer le DataFrame de comparaison
- 📝 Ajouter les lignes problématiques, les moyennes avant/après et les moyennes globales dans un DataFrame unique (`comparison_df`).

#### 🔹 Étape 5 : Calculer les différences absolues
- ➖ Différences entre chaque ligne et les moyennes avant/après pour les colonnes numériques.
- 🔗 Concaténer ces différences au DataFrame principal pour visualiser les écarts.

#### 🔹 Étape 6 : Affichage final
- 👀 Afficher uniquement les colonnes problématiques et la colonne `time`.
- 🖥️ Fournir une vue claire de la variation par rapport aux contextes global et local pour chaque minute analysée.

Cette analyse fournit un **diagnostic précis** des anomalies et prépare le DataFrame pour d’éventuelles corrections ou décisions.


In [ ]:
if 'df_cleaned' not in locals() or 'df_to_inspect' not in locals():
    print("Erreur : Les DataFrames 'df_cleaned' ou 'df_to_inspect' n'ont pas été trouvés.")
else:
    numeric_cols = df_cleaned.select_dtypes(include=np.number).columns.drop('time_minute', errors='ignore')

    # =============================================================================
    # === NOUVELLE ÉTAPE : CALCULER LES MOYENNES GLOBALES UNE SEULE FOIS ===
    # =============================================================================
    # On le fait avant la boucle pour être plus efficace
    global_means = df_cleaned[numeric_cols].mean()
    global_means.name = 'moyenne_globale_colonne'
    # =============================================================================

    grouped_problems = df_to_inspect.groupby('time_minute')
    print(f"Début de l'analyse contextuelle pour {len(grouped_problems)} groupes problématiques...")

    for minute, group in grouped_problems:
        if len(group) != 2:
            print(f"\n--- Le groupe à {minute} a {len(group)} lignes, il est ignoré. ---")
            continue

        # --- Étapes 1 à 3 (inchangées) ---
        ligne1 = group.iloc[0]
        ligne2 = group.iloc[1]
        pos1 = df_cleaned.index.get_loc(ligne1.name)
        pos2 = df_cleaned.index.get_loc(ligne2.name)

        start_before = max(0, pos1 - 10)
        df_before = df_cleaned.iloc[start_before:pos1]
        moyenne_avant = df_before[numeric_cols].mean() if not df_before.empty else pd.Series(np.nan, index=numeric_cols)
        moyenne_avant.name = 'moyenne_10_avant'

        start_after = pos2 + 1
        end_after = min(len(df_cleaned), start_after + 10)
        df_after = df_cleaned.iloc[start_after:end_after]
        moyenne_apres = df_after[numeric_cols].mean() if not df_after.empty else pd.Series(np.nan, index=numeric_cols)
        moyenne_apres.name = 'moyenne_10_apres'

        # --- MODIFICATION DE L'ÉTAPE 4 ---
        ligne1.name = f"ligne_{ligne1.name} (problem)"
        ligne2.name = f"ligne_{ligne2.name} (problem)"
        # On ajoute la ligne 'global_means' au début du DataFrame de comparaison
        comparison_df = pd.DataFrame([global_means, moyenne_avant, ligne1, ligne2, moyenne_apres])

        # --- Étape 5 (inchangée) : Calculer les différences ---
        ligne1_numeric = ligne1[numeric_cols]
        ligne2_numeric = ligne2[numeric_cols]
        diff_l1_vs_avant = abs(ligne1_numeric - moyenne_avant); diff_l1_vs_avant.name = 'abs_diff(L1-Avant)'
        diff_l2_vs_avant = abs(ligne2_numeric - moyenne_avant); diff_l2_vs_avant.name = 'abs_diff(L2-Avant)'
        diff_l1_vs_apres = abs(ligne1_numeric - moyenne_apres); diff_l1_vs_apres.name = 'abs_diff(L1-Apres)'
        diff_l2_vs_apres = abs(ligne2_numeric - moyenne_apres); diff_l2_vs_apres.name = 'abs_diff(L2-Apres)'
        diff_df = pd.DataFrame([diff_l1_vs_avant, diff_l2_vs_avant, diff_l1_vs_apres, diff_l2_vs_apres])

        final_comparison_df = pd.concat([comparison_df, diff_df])

        # --- Étape 6 (inchangée) : Affichage ---
        problem_cols = ligne1['problem_columns']
        display_cols = ['time'] + problem_cols
        display_cols = [col for col in display_cols if col in final_comparison_df.columns]

        print("\n" + "="*80)
        print(f"ANALYSE DU GROUPE À {minute.strftime('%Y-%m-%d %H:%M')}")
        print(f"Colonnes avec variation significative : {problem_cols}")
        print("="*80)

        pd.set_option('display.float_format', '{:.2f}'.format)
        display(final_comparison_df[display_cols])

### ⚙️ Correction automatique et analyse des cas non résolus

Cette étape combine deux phases principales pour gérer les **doublons de mesures par minute**.

---

#### 🔹 Phase 1 : Correction automatique
- 🟢 Objectif : Identifier automatiquement la ligne à supprimer lorsque deux enregistrements présentent une variation minime par rapport au contexte.
- ⚙️ Calcul de :
  - Moyennes globales des colonnes numériques (`global_means`).
  - Moyennes locales sur 10 lignes avant et 5 lignes après chaque groupe problématique.
- 📌 Critère de suppression :
  - Si la différence minimale entre une ligne et le contexte est inférieure à 30 % de la moyenne globale pour cette colonne.
- 🗑️ Les lignes identifiées sont supprimées de `df_corrected`.

---

#### 🔹 Phase 2 : Affichage détaillé des cas non résolus
- 🔍 Objectif : Analyser les groupes qui restent problématiques après la correction automatique.
- 📊 Pour chaque groupe restant :
  - Calcul des moyennes locales avant/après.
  - Affichage des lignes problématiques et des colonnes présentant des variations significatives.
  - Comparaison avec la moyenne globale pour repérer les anomalies.
- 👀 Résultat :
  - Identification précise des lignes à examiner manuellement.
  - Vue contextualisée pour chaque minute, facilitant la prise de décision.

---

✅ Cette double approche permet de **corriger automatiquement la majorité des doublons** tout en offrant un diagnostic clair pour les cas nécessitant une intervention manuelle.


In [ ]:
if 'df_cleaned' not in locals() or 'df_to_inspect' not in locals():
    print("Erreur : Les DataFrames requis n'ont pas été trouvés.")
else:
    # --- Définitions communes ---
    marge_diff = 0.3
    numeric_cols = df_cleaned.select_dtypes(include=np.number).columns.drop('time_minute', errors='ignore')
    global_means = df_cleaned[numeric_cols].mean()

    # =============================================================================
    # === PHASE 1 : CORRECTION AUTOMATIQUE ===
    # =============================================================================
    df_corrected = df_cleaned.copy()
    grouped_problems_for_correction = df_to_inspect.groupby('time_minute')
    indexes_to_delete = []

    print("--- PHASE 1: Début du processus de correction automatique ---")
    for minute, group in grouped_problems_for_correction:
        if len(group) != 2: continue

        ligne1, original_index1 = group.iloc[0], group.iloc[0].name
        ligne2, original_index2 = group.iloc[1], group.iloc[1].name
        pos1 = df_cleaned.index.get_loc(original_index1)
        pos2 = df_cleaned.index.get_loc(original_index2)

        moyenne_avant = df_cleaned.iloc[max(0, pos1 - 10):pos1][numeric_cols].mean()
        moyenne_apres = df_cleaned.iloc[pos2 + 1:min(len(df_cleaned), pos2 + 6)][numeric_cols].mean()

        diff_df = pd.DataFrame({
            'abs_diff(L1-Avant)': abs(ligne1[numeric_cols] - moyenne_avant),
            'abs_diff(L2-Avant)': abs(ligne2[numeric_cols] - moyenne_avant),
            'abs_diff(L1-Apres)': abs(ligne1[numeric_cols] - moyenne_apres),
            'abs_diff(L2-Apres)': abs(ligne2[numeric_cols] - moyenne_apres)
        }).transpose()

        problem_cols = ligne1['problem_columns']
        problem_cols_in_diff = [col for col in problem_cols if col in diff_df.columns]
        if not problem_cols_in_diff: continue

        unstacked_diffs = diff_df[problem_cols_in_diff].unstack()
        min_loc = unstacked_diffs.idxmin()
        min_col_name, min_row_name = min_loc
        min_diff_value = unstacked_diffs[min_loc]

        threshold_value = marge_diff * global_means[min_col_name]

        if min_diff_value < threshold_value:
            if 'L1' in min_row_name: indexes_to_delete.append(original_index2)
            else: indexes_to_delete.append(original_index1)

    if indexes_to_delete:
        print(f"--- {len(indexes_to_delete)} lignes ont été identifiées pour suppression. ---")
        df_corrected.drop(index=indexes_to_delete, inplace=True)
    else:
        print("--- Aucune ligne n'a été supprimée automatiquement. ---")

    # =============================================================================
    # === PHASE 2 : AFFICHAGE DÉTAILLÉ DES CAS NON RÉSOLUS ===
    # =============================================================================
    print("\n" + "="*80)
    print("PHASE 2: ANALYSE DÉTAILLÉE DES CAS NON RÉSOLUS")
    print("="*80)

    # Identifier les minutes qui posent encore problème DANS LE DATAFRAME CORRIGÉ
    df_corrected['time_minute'] = df_corrected['time'].dt.floor('min')
    final_counts = df_corrected['time_minute'].value_counts()
    remaining_problems_series = final_counts[final_counts > 1]

    if remaining_problems_series.empty:
        print("✅ Tous les cas de doublons ont été résolus automatiquement !")
    else:
        print(f"{len(remaining_problems_series)} minute(s) contiennent encore des enregistrements multiples.")

        # Pour l'analyse, on retourne chercher les informations complètes dans df_to_inspect
        remaining_to_inspect = df_to_inspect[df_to_inspect['time_minute'].isin(remaining_problems_series.index)]
        grouped_remaining_for_display = remaining_to_inspect.groupby('time_minute')

        for minute, group in grouped_remaining_for_display:
            if len(group) != 2: continue

            # Ici, on utilise le code d'affichage que vous avez fourni, qui fonctionne
            ligne1 = group.iloc[0]
            ligne2 = group.iloc[1]
            pos1 = df_cleaned.index.get_loc(ligne1.name)
            pos2 = df_cleaned.index.get_loc(ligne2.name)

            start_before = max(0, pos1 - 10)
            df_before = df_cleaned.iloc[start_before:pos1]
            moyenne_avant = df_before[numeric_cols].mean() if not df_before.empty else pd.Series(np.nan, index=numeric_cols)
            moyenne_avant.name = 'moyenne_10_avant'

            start_after = pos2 + 1
            end_after = min(len(df_cleaned), start_after + 10)
            df_after = df_cleaned.iloc[start_after:end_after]
            moyenne_apres = df_after[numeric_cols].mean() if not df_after.empty else pd.Series(np.nan, index=numeric_cols)
            moyenne_apres.name = 'moyenne_10_apres'

            global_means.name = 'moyenne_globale_colonne'
            ligne1.name = f"ligne_{ligne1.name} (problem)"
            ligne2.name = f"ligne_{ligne2.name} (problem)"
            comparison_df = pd.DataFrame([global_means, moyenne_avant, ligne1, ligne2, moyenne_apres])

            ligne1_numeric = ligne1[numeric_cols]
            ligne2_numeric = ligne2[numeric_cols]
            diff_l1_vs_avant = abs(ligne1_numeric - moyenne_avant); diff_l1_vs_avant.name = 'abs_diff(L1-Avant)'
            diff_l2_vs_avant = abs(ligne2_numeric - moyenne_avant); diff_l2_vs_avant.name = 'abs_diff(L2-Avant)'
            diff_l1_vs_apres = abs(ligne1_numeric - moyenne_apres); diff_l1_vs_apres.name = 'abs_diff(L1-Apres)'
            diff_l2_vs_apres = abs(ligne2_numeric - moyenne_apres); diff_l2_vs_apres.name = 'abs_diff(L2-Apres)'
            diff_df = pd.DataFrame([diff_l1_vs_avant, diff_l2_vs_avant, diff_l1_vs_apres, diff_l2_vs_apres])

            final_comparison_df = pd.concat([comparison_df, diff_df])

            # Cette partie est maintenant sûre car 'ligne1' vient de 'df_to_inspect'
            problem_cols = ligne1['problem_columns']
            display_cols = ['time'] + problem_cols
            display_cols = [col for col in display_cols if col in final_comparison_df.columns]

            print("\n" + "-"*80)
            print(f"ANALYSE DU GROUPE À {minute.strftime('%Y-%m-%d %H:%M')}")
            print(f"Colonnes avec variation significative : {problem_cols}")
            print("-"*80)
            pd.set_option('display.float_format', '{:.2f}'.format)
            display(final_comparison_df[display_cols])

### 🌊 Lissage final et rapport des doublons restants

Cette étape vise à traiter les **cas restants non résolus** après les phases de correction automatique.

---

#### 🔹 Phase 3 : Tentative de lissage
- 🎯 Objectif : Fusionner les lignes problématiques lorsque le **contexte avant/après est stable**.
- 📝 Méthodologie :
  - Calcul des moyennes locales **avant** et **après** chaque minute problématique.
  - Vérification de la stabilité du contexte : différence entre moyennes < 20 % de la moyenne globale (`marge * global_means`).
  - Si stable :
    - Les deux lignes sont fusionnées.
    - Valeur de chaque colonne problématique remplacée par la moyenne du contexte.
    - Suppression de la ligne redondante.
  - Si instable :
    - Les deux lignes sont conservées pour éviter de déformer les données.

- ✅ Bénéfice : réduit manuellement les doublons tout en respectant la cohérence temporelle.

---

#### 🔹 Phase 4 : Rapport final
- 📊 Vérification du nombre de minutes restant avec des doublons après lissage.
- ✅ Résultat attendu : **tous les doublons corrigés ou lissés**, sauf quelques cas complexes à conserver pour analyse.
- ⚠️ Les cas complexes sont affichés pour examen manuel.

---

💡 Cette approche en plusieurs phases permet un **nettoyage très fin** des données temporelles tout en préservant l'intégrité des mesures.


In [ ]:
if 'df_cleaned' not in locals() or 'df_to_inspect' not in locals():
    print("Erreur : Les DataFrames requis n'ont pas été trouvés.")
else:
    # =============================================================================
    # === NOUVELLE PHASE 3 : LISSAGE DES CAS RESTANTS ===
    # =============================================================================
    if not remaining_problems_series.empty:
        print("\n" + "="*80)
        print("PHASE 3: TENTATIVE DE LISSAGE FINAL DES CAS RESTANTS")
        print("="*80)

        updates_to_make = []
        indexes_to_delete_phase3 = []

        remaining_to_inspect = df_to_inspect[df_to_inspect['time_minute'].isin(remaining_problems_series.index)]
        grouped_remaining_for_smoothing = remaining_to_inspect.groupby('time_minute')

        for minute, group in grouped_remaining_for_smoothing:
            if len(group) != 2: continue

            ligne1 = group.iloc[0]
            ligne2 = group.iloc[1]
            pos1 = df_cleaned.index.get_loc(ligne1.name)
            pos2 = df_cleaned.index.get_loc(ligne2.name)
            moyenne_avant = df_cleaned.iloc[max(0, pos1 - 10):pos1][numeric_cols].mean()
            moyenne_apres = df_cleaned.iloc[pos2 + 1:min(len(df_cleaned), pos2 + 11)][numeric_cols].mean()

            is_context_stable = True
            problem_cols = ligne1['problem_columns']
            for col in problem_cols:
                if col not in numeric_cols: continue
                mycontext_diff = abs(moyenne_avant[col] - moyenne_apres[col])
                mythreshold = marge * global_means[col]
                if mycontext_diff > mythreshold:
                    is_context_stable = False
                    break

            if is_context_stable:
                print(f"Groupe à {minute.strftime('%H:%M')}: CONTEXTE STABLE -> Lissage et fusion des lignes.")
                index_to_update = group.index[0]
                index_to_delete = group.index[1]
                indexes_to_delete_phase3.append(index_to_delete)
                for col in problem_cols:
                    if col not in numeric_cols: continue
                    new_value = (moyenne_avant[col] + moyenne_apres[col]) / 2
                    updates_to_make.append((index_to_update, col, new_value))
            else:
                print(f"Groupe à {minute.strftime('%H:%M')}: CONTEXTE INSTABLE -> Conservation des deux lignes.")

        if updates_to_make:
            for idx, col, val in updates_to_make:
                df_corrected.loc[idx, col] = val

        if indexes_to_delete_phase3:
            df_corrected.drop(index=indexes_to_delete_phase3, inplace=True)
            print(f"\n--- {len(indexes_to_delete_phase3)} ligne(s) ont été lissées et fusionnées. ---")

    # =============================================================================
    # === PHASE 4 : RAPPORT FINAL ===
    # =============================================================================
    print("\n" + "="*80)
    print("PHASE 4: RAPPORT FINAL APRÈS LISSAGE")
    print("="*80)

    df_corrected['time_minute'] = df_corrected['time'].dt.floor('min')
    final_final_counts = df_corrected['time_minute'].value_counts()
    truly_remaining_problems = final_final_counts[final_final_counts > 1]

    if truly_remaining_problems.empty:
        print("✅ Tous les cas de doublons ont été résolus par correction ou lissage.")
    else:
        print(f"⚠️ {len(truly_remaining_problems)} cas complexes subsistent et ont été conservés :")
        display(df_corrected[df_corrected['time_minute'].isin(truly_remaining_problems.index)])

### 🔍 Vérification finale avant indexation

Cette étape permet de **contrôler qu’aucune duplication à la minute ne subsiste** avant d’indexer le DataFrame.

---

#### 🔹 Étape 1 : Identifier les minutes avec doublons
- ⏱️ Création d'une colonne temporaire `time_minute` (temps tronqué à la minute).
- 📊 Comptage du nombre d’occurrences pour chaque minute.
- ⚠️ Filtrage pour ne garder que les minutes contenant **plus d’un enregistrement**.

---

#### 🔹 Étape 2 : Affichage des lignes problématiques
- ✅ Si aucune duplication n’est trouvée : prêt pour l’indexation et la sauvegarde.
- ⚠️ Si des doublons persistent :
  - Affichage détaillé des minutes et lignes concernées.
  - Les cas sont conservés car leur variation a été jugée **significative**.
  - Classement par minute pour une lecture facile.
  - Toutes les colonnes visibles pour l’analyse.

---

#### 🔹 Étapes possibles pour gérer les doublons restants
1. **Agrégation par moyenne** : solution simple pour obtenir un index unique.
   Exemple :
   `df_final = df_corrected.groupby(df_corrected['time'].dt.floor('min')).mean()`
2. **Suppression manuelle** : retirer les lignes clairement erronées.
3. **Conservation pour analyse spécifique** : certains événements peuvent nécessiter un traitement séparé.

💡 Cette étape est cruciale pour garantir l’**intégrité temporelle** avant toute manipulation ou sauvegarde finale.


In [ ]:
if 'df_corrected' not in locals():
    print("Erreur : Le DataFrame 'df_corrected' n'a pas été trouvé.")
else:
    print("Vérification finale du DataFrame 'df_corrected' avant indexation...")

    # Créer une copie pour l'analyse
    df_analysis = df_corrected.copy()

    # =============================================================================
    # === ÉTAPE 1 : IDENTIFIER LES MINUTES CONTENANT DES DOUBLONS ===
    # =============================================================================

    # Créer une colonne temporaire avec le temps tronqué à la minute
    df_analysis['time_minute'] = df_analysis['time'].dt.floor('min')

    # Compter le nombre d'occurrences de chaque minute
    minute_counts = df_analysis['time_minute'].value_counts()

    # Filtrer pour ne garder que les minutes qui apparaissent plus d'une fois
    remaining_duplicates_minutes = minute_counts[minute_counts > 1].index

    # =============================================================================
    # === ÉTAPE 2 : AFFICHER LES LIGNES CORRESPONDANTES ===
    # =============================================================================

    if remaining_duplicates_minutes.empty:
        print("\n✅ BONNE NOUVELLE : Aucune duplication à la minute n'a été trouvée.")
        print("Vous pouvez procéder directement à l'indexation et à la sauvegarde.")
    else:
        print("\n" + "="*80)
        print("ATTENTION : LES LIGNES SUIVANTES PROVOQUERONT UN INDEX NON UNIQUE")
        print("="*80)
        print(f"{len(remaining_duplicates_minutes)} minute(s) contiennent encore plusieurs enregistrements.")
        print("Ces cas ont été conservés par votre algorithme car leur variation a été jugée 'significative'.")

        # Filtrer le DataFrame pour n'afficher QUE les lignes qui posent problème
        df_to_display = df_analysis[df_analysis['time_minute'].isin(remaining_duplicates_minutes)]

        # Trier pour que les groupes soient bien visibles
        df_to_display.sort_values(by=['time_minute', 'time'], inplace=True)

        # Configurer l'affichage pour voir toutes les colonnes
        pd.set_option('display.max_columns', None)
        pd.set_option('display.width', 1000)

        # Afficher le DataFrame final
        display(df_to_display)

        print("\n--- Analyse et Prochaines Étapes ---")
        print("Vous devez maintenant décider comment gérer ces cas restants :")
        print("1. Agréger par la moyenne : C'est la solution la plus simple pour obtenir un index unique.")
        print("   Ex: df_final = df_corrected.groupby(df_corrected['time'].dt.floor('min')).mean()")
        print("2. Suppression manuelle : Si une ligne est clairement une erreur, vous pouvez la supprimer par son index.")
        print("3. Conserver pour une analyse spécifique : Peut-être que ces événements ne doivent pas être dans le jeu de données principal, mais analysés séparément.")

### ⏱️ Vérification des minutes manquantes dans l’index temporel

Cette étape permet de s'assurer que le **jeu de données est continu minute par minute** après nettoyage et indexation.

---

#### 🔹 Étape 1 : Chargement des données
- 📂 Fichier : `data_final_cleaned_indexed.csv`
- ✅ Chargement avec `index_col=0` pour utiliser la colonne temporelle comme index.
- ℹ️ Affichage de l’intervalle temporel couvert par le jeu de données : du premier au dernier timestamp.

---

#### 🔹 Étape 2 : Vérification de la continuité
1. 📏 Création d’un **index de référence attendu** :
   - Début : première minute
   - Fin : dernière minute
   - Fréquence : 1 minute (`freq='min'`)
2. 🔍 Identification des timestamps **manquants** dans le DataFrame par rapport à l’index de référence.
3. 📊 Rapport :
   - ✅ Si aucune minute manquante : le jeu de données est complet.
   - ⚠️ Si des minutes manquent :
     - Nombre total de minutes manquantes.
     - Pourcentage de données manquantes.
     - Aperçu des premières minutes manquantes (jusqu’à 10) pour inspection rapide.

---

💡 Cette étape est cruciale pour garantir que **l’analyse chronologique ou les calculs par minute** ne seront pas faussés par des interruptions dans la série temporelle.


In [ ]:
# --- Étape 1 : Chargement des données (comme vous l'avez fait) ---

file_path = 'data_final_cleaned_indexed.csv'
try:
    df = pd.read_csv(file_path, index_col=0, parse_dates=True)
    print("Fichier CSV chargé avec succès.")
except FileNotFoundError:
    print(f"Erreur : Le fichier '{file_path}' n'a pas été trouvé.")
    exit()

print(f"\nLe jeu de données s'étend de {df.index.min()} à {df.index.max()}.")

# =============================================================================
# === ÉTAPE 2 : VÉRIFICATION DES MINUTES MANQUANTES ===
# =============================================================================

print("\n" + "="*80)
print("VÉRIFICATION DES MINUTES MANQUANTES DANS L'INDEX TEMPOREL")
print("="*80)

# 1. Créer l'index de référence attendu
#    Il commence au premier timestamp, se termine au dernier, avec une fréquence d'une minute ('min').
expected_index = pd.date_range(start=df.index.min(), end=df.index.max(), freq='min')

# 2. Trouver les timestamps qui sont dans l'index de référence mais PAS dans l'index de notre DataFrame
missing_timestamps = expected_index.difference(df.index)

# 3. Afficher le rapport
if missing_timestamps.empty:
    print("✅ BONNE NOUVELLE : Aucune minute manquante n'a été détectée dans l'intervalle.")
    print("Votre jeu de données a une fréquence temporelle continue.")
else:
    print(f"⚠️ ATTENTION : {len(missing_timestamps)} minutes manquantes ont été détectées.")

    total_expected_minutes = len(expected_index)
    percentage_missing = (len(missing_timestamps) / total_expected_minutes) * 100

    print(f"   - Période totale attendue : {total_expected_minutes} minutes.")
    print(f"   - Pourcentage de données manquantes : {percentage_missing:.2f}%")

    print("\nVoici un aperçu des 10 premières minutes manquantes :")
    # On affiche seulement les 10 premières pour ne pas surcharger l'affichage
    for ts in missing_timestamps[:10]:
        print(f"   - {ts}")

### 🕵️ Analyse des périodes de minutes manquantes consécutives

Cette étape permet d’identifier **les blocs de minutes manquantes** et leur durée, plutôt que de lister chaque minute isolée.

---

#### 🔹 Étape 1 : Vérification de l’existence de minutes manquantes
- Si aucune minute n’est manquante : message `Aucune minute manquante à analyser.`
- Sinon : passage à l’analyse des périodes consécutives.

---

#### 🔹 Étape 2 : Identification des périodes manquantes
1. 🔹 Initialisation de la première période avec le premier timestamp manquant.
2. 🔹 Parcours de tous les timestamps manquants :
   - Si le timestamp actuel **n’est pas consécutif** au précédent, la période précédente est terminée.
   - Enregistrer le **début**, la **fin**, et la **durée** de la période.
   - Démarrer une **nouvelle période**.
3. 🔹 Après la boucle, ajouter la dernière période manquante restante.

---

#### 🔹 Étape 3 : Préparer les résultats pour affichage
- Conversion de la liste des périodes en **DataFrame** (`gaps_df`).
- Conversion de la durée en **minutes entières** pour plus de clarté.
- Tri par **durée décroissante** pour mettre en évidence les plus longues interruptions.
- Réinitialisation de l’index pour un affichage propre.

---

#### 🔹 Étape 4 : Interprétation
- 📊 Chaque ligne du DataFrame contient :
  - `start` : début de la période manquante
  - `end` : fin de la période manquante
  - `duration_minutes` : durée totale en minutes
- 📝 Ce tableau permet de prioriser les **périodes critiques à combler ou analyser**.


In [ ]:
# --- Ce code s'exécute si la condition 'if not missing_timestamps.empty:' est vraie ---

# S'assurer que la variable 'missing_timestamps' existe depuis la cellule précédente.
if 'missing_timestamps' not in locals() or missing_timestamps.empty:
    print("Aucune minute manquante à analyser.")
else:
    print("\n" + "="*80)
    print("ANALYSE DES PÉRIODES DE MINUTES MANQUANTES CONSÉCUTIVES")
    print("="*80)

    # Liste pour stocker les informations sur chaque période (gap)
    gaps = []

    # Démarrer la première période avec le premier timestamp manquant
    start_of_gap = missing_timestamps[0]
    previous_ts = missing_timestamps[0]

    # Parcourir tous les timestamps manquants à partir du deuxième
    for ts in missing_timestamps[1:]:
        # Vérifier si le timestamp actuel n'est PAS la minute qui suit la précédente
        if ts != previous_ts + pd.Timedelta(minutes=1):
            # Si c'est le cas, la période précédente est terminée.
            end_of_gap = previous_ts
            # Calculer la durée de la période
            duration = (end_of_gap - start_of_gap) + pd.Timedelta(minutes=1)
            # Ajouter les informations à notre liste
            gaps.append({'start': start_of_gap, 'end': end_of_gap, 'duration_minutes': duration.total_seconds() / 60})

            # Démarrer une nouvelle période
            start_of_gap = ts

        # Mettre à jour le timestamp précédent pour la prochaine itération
        previous_ts = ts

    # --- Important : Ajouter la toute dernière période après la fin de la boucle ---
    end_of_gap = missing_timestamps[-1]
    duration = (end_of_gap - start_of_gap) + pd.Timedelta(minutes=1)
    gaps.append({'start': start_of_gap, 'end': end_of_gap, 'duration_minutes': duration.total_seconds() / 60})

    # --- Afficher les résultats de manière lisible ---

    # Convertir la liste de résultats en DataFrame pour un affichage et un tri faciles
    gaps_df = pd.DataFrame(gaps)

    # Convertir la durée en entier pour plus de clarté
    gaps_df['duration_minutes'] = gaps_df['duration_minutes'].astype(int)

    # Trier par durée pour voir les plus longues périodes manquantes en premier
    gaps_df.sort_values(by='duration_minutes', ascending=False, inplace=True)

    # Réinitialiser l'index pour un affichage propre
    gaps_df.reset_index(drop=True, inplace=True)

    print(f"Les {len(missing_timestamps)} minutes manquantes sont réparties en {len(gaps_df)} périodes consécutives :")
    display(gaps_df)

### 🛠️ Rééchantillonnage pour rendre les minutes manquantes explicites

Cette étape permet de **mettre en évidence les minutes manquantes** en créant des lignes avec `NaN` dans le DataFrame.

---

#### 🔹 Étape 1 : Chargement des données
- Lecture du fichier CSV `data_final_cleaned_indexed.csv`
- Indexation sur la colonne temporelle existante (`time`).

---

#### 🔹 Étape 2 : Création d’un index temporel complet
- On crée un **index parfait** à fréquence 1 minute (`freq='min'`) couvrant toute la période.
- Réindexation du DataFrame avec cet index :
  - Les minutes **manquantes** sont automatiquement remplies par des lignes contenant `NaN`.

---

#### 🔹 Étape 3 : Vérification
- ✅ Taille du DataFrame avant et après rééchantillonnage.
- 📊 Nombre de lignes contenant au moins un `NaN` = nombre de minutes manquantes.
- Objectif : préparer le jeu de données pour un **nettoyage ou interpolation ultérieure**.


In [ ]:
import pandas as pd

# --- Étape 1 : Chargement des données ---
file_path = 'data_final_cleaned_indexed.csv'
df = pd.read_csv(file_path, index_col=0, parse_dates=True)

# =============================================================================
# === ÉTAPE 2 : RÉÉCHANTILLONNAGE POUR RENDRE LES TROUS EXPLICITES AVEC NaN ===
# =============================================================================
print("\n--- Création d'un index temporel complet ---")

# 1. Créer l'index de référence parfait, sans aucun trou
full_index = pd.date_range(start=df.index.min(), end=df.index.max(), freq='min')

# 2. Ré-indexer le DataFrame. C'est l'étape clé.
#    Pandas va automatiquement créer des lignes remplies de NaN pour toutes les minutes manquantes.
df_complete = df.reindex(full_index)

print(f"Taille du DataFrame avant rééchantillonnage : {len(df)}")
print(f"Taille du DataFrame après rééchantillonnage : {len(df_complete)}")

# Compter le nombre de lignes qui contiennent au moins un NaN
# Ce nombre devrait être égal au nombre de minutes manquantes que vous aviez trouvé (2880)
num_rows_with_nan = df_complete.isnull().any(axis=1).sum()
print(f"{num_rows_with_nan} lignes avec des valeurs manquantes (NaN) ont été créées, ce qui est attendu.")


### 🏁 Finalisation et exportation du DataFrame complet

Cette cellule prépare le DataFrame **rééchantillonné et complété** pour l'analyse ou l'export final.

---

#### 🔹 Étape 1 : Copie de travail
- `df_complete` est copié dans `df_final` pour sécuriser les manipulations finales.

---

#### 🔹 Étape 2 : Vérification de l'index
- ✅ Vérifie que l'index temporel est **unique** après le rééchantillonnage.
- ⚠️ Une anomalie ici indiquerait un problème inattendu dans l'index.

---

#### 🔹 Étape 3 : Vérification des valeurs manquantes
- Comptabilise les lignes avec au moins un `NaN`.
- Ces lignes correspondent aux **minutes initialement manquantes**, désormais explicites pour analyse/interpolation.

---

#### 🔹 Étape 4 : Exportation
- Le DataFrame final est enregistré en CSV (`data_final_cleaned_indexed.csv`) avec l’index temporel conservé.
- ✅ Permet de disposer d’un jeu de données **complet et prêt pour l’analyse chronologique**.


In [ ]:
# =============================================================================
# === NOUVELLE CELLULE : FINALISATION ET EXPORTATION DU DATAFRAME COMPLET ===
# =============================================================================

# --- Pré-requis : S'assurer que 'df_complete' est disponible depuis la cellule précédente ---
if 'df_complete' not in locals():
    print("❌ Erreur : Le DataFrame 'df_complete' n'a pas été trouvé.")
    print("Veuillez d'abord exécuter la cellule de rééchantillonnage.")
else:
    print("Préparation du DataFrame complet pour l'enregistrement...")

    # --- Étape 1 : Copie de travail ---
    # La variable df_complete est déjà presque parfaite, on la copie pour la finalisation.
    df_final = df_complete.copy()

    # --- Étape 2 : Vérification de l'index ---
    # Cette étape est toujours une bonne pratique.
    print("\nVérification de l'index...")
    if df_final.index.is_unique:
        print("✅ L'index temporel est unique, comme attendu après le rééchantillonnage.")
    else:
        print("⚠️ Erreur inattendue : L'index n'est pas unique, ce qui ne devrait pas arriver.")

    # --- Étape 3 : Vérification des données manquantes ---
    # C'est une vérification utile à ajouter ici
    num_rows_with_nan = df_final.isnull().any(axis=1).sum()
    if num_rows_with_nan > 0:
        print(f"✅ Le DataFrame contient {num_rows_with_nan} lignes avec des NaN, représentant les minutes manquantes.")
    else:
        print("Le DataFrame ne contient aucune valeur manquante.")

    print("\nAperçu du DataFrame final :")
    display(df_final.head())

    # --- Étape 4 : Enregistrement dans un fichier CSV ---
    output_file_path = 'data_final_cleaned_indexed.csv'

    # Enregistrer le DataFrame en CSV.
    # L'index sera automatiquement sauvegardé car il est l'index du DataFrame.
    try:
        df_final.to_csv(output_file_path)
        print(f"\n✅ Le DataFrame final a été enregistré avec succès dans le fichier : '{output_file_path}'")
    except Exception as e:
        print(f"\n❌ Une erreur est survenue lors de l'enregistrement du fichier : {e}")

## ✅ Conclusion du notebook

Le DataFrame a été **nettoyé, fusionné, lissé** et **réindexé avec toutes les minutes explicites**.
Les doublons significatifs ont été traités et les minutes manquantes sont désormais représentées par des `NaN`, prêtes pour toute interpolation ou analyse temporelle.

Vous pouvez maintenant passer au **prochain notebook** : [`data_cleaned_indexed.ipynb`](./data_cleaned_indexed.ipynb) pour continuer l'analyse ou la préparation des données.
